In [2]:
# Set wd to STB_social_media_analytics.
%cd /home/zyf0717/git/STB_social_media_analytics/

/home/zyf0717/git/STB_social_media_analytics


In [3]:
import pandas as pd
from pandas.io.json import json_normalize

from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions

# Setup the API caller
authenticator = IAMAuthenticator('bkC6hyTVS4qonY34vS0d8Q5F2YY4Kz9m0fyj1cN-5VFw')
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2019-07-12',
    authenticator=authenticator
)
natural_language_understanding.set_service_url(
    'https://gateway.watsonplatform.net/natural-language-understanding/api/v1/analyze?version=2019-07-12')

# Open CSV containing reviews to score
folder_name = '200125_142611'
poi_index = 1
reviews_df = pd.read_csv('./tripadvisor/finalised_output/{}/reviews/{}.csv'.format(folder_name, poi_index))

# Create CSVs
keywords_col_names = ['WEBSITE_ID', 'REVIEW_INDEX', 'TEXT', 'RELEVANCE', 'COUNT',
                     'SENTIMENT_SCORE', 'SENTIMENT_LABEL', 'SADNESS', 'JOY',
                     'FEAR', 'DISGUST', 'ANGER']
pd.DataFrame(columns=keywords_col_names).to_csv(
    './tripadvisor/finalised_output/{}/reviews/{}_keywords.csv'.format(
        folder_name,
        poi_index),
    mode='a',
    header=False,
    index=False
)
entities_col_names = ['WEBSITE_ID', 'REVIEW_INDEX', 'TYPE', 'TEXT', 'RELEVANCE',
                      'COUNT', 'CONFIDENCE', 'SENTIMENT_SCORE',
                      'SENTIMENT_LABEL', 'SADNESS', 'JOY', 'FEAR', 'DISGUST',
                      'ANGER', 'DISAMBIGUATION_NAME', 'DISAMBIGUATION_RESOURCE']
pd.DataFrame(columns=entities_col_names).to_csv(
    './tripadvisor/finalised_output/{}/reviews/{}_entities.csv'.format(
        folder_name,
        poi_index),
    mode='a',
    header=False,
    index=False
)

keywords_col_names_map = {
    'text': 'TEXT',
    'relevance': 'RELEVANCE',
    'count': 'COUNT',
    'sentiment.score': 'SENTIMENT_SCORE',
    'sentiment.label': 'SENTIMENT_LABEL',
    'emotion.sadness': 'SADNESS',
    'emotion.joy': 'JOY',
    'emotion.fear': 'FEAR',
    'emotion.disgust': 'DISGUST',
    'emotion.anger': 'ANGER'
}

entities_col_names_map = {
    'type': 'TYPE',
    'text': 'TEXT',
    'relevance': 'RELEVANCE',
    'count': 'COUNT',
    'confidence': 'CONFIDENCE',
    'sentiment.score': 'SENTIMENT_SCORE',
    'sentiment.label': 'SENTIMENT_LABEL',
    'emotion.sadness': 'SADNESS',
    'emotion.joy': 'JOY',
    'emotion.fear': 'FEAR',
    'emotion.disgust': 'DISGUST',
    'emotion.anger': 'ANGER',
    'sentiment.mixed': 'MIXED_SENTIMENT',
    'disambiguation.name': 'DISAMBIGUATION_NAME',
    'disambiguation.dbpedia_resource': 'DISAMBIGUATION_RESOURCE'
}

# Making the API calls
while len(reviews_df.index) > 0:
    text = reviews_df.iloc[0][9]
    print(text)
    response = natural_language_understanding.analyze(
        text=text,
        features=Features(
            entities=EntitiesOptions(emotion=True, sentiment=True, limit=1000),
            keywords=KeywordsOptions(emotion=True, sentiment=True, limit=1000)
        )
    ).get_result()

    keywords_df = json_normalize(response['keywords']).rename(columns=keywords_col_names_map)
    keywords_df.insert(0, 'REVIEW_INDEX', reviews_df.iloc[0][4])
    keywords_df.insert(0, 'WEBSITE_ID', 1)

    keywords_df.to_csv(
        './tripadvisor/finalised_output/{}/reviews/{}_keywords.csv'.format(
            folder_name,
            poi_index),
        mode='a',
        header=False,
        index=False
    )

    entities_df = json_normalize(response['entities']).rename(columns=entities_col_names_map)
    entities_df.insert(0, 'REVIEW_INDEX', reviews_df.iloc[0][4])
    entities_df.insert(0, 'WEBSITE_ID', 1)

    entities_df.to_csv(
        './tripadvisor/finalised_output/{}/reviews/{}_keywords.csv'.format(
            folder_name,
            poi_index),
        mode='a',
        header=False,
        index=False
    )

    reviews_df = reviews_df.iloc[1:]

01-2020


ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/home/zyf0717/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 157, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: c88d533399fbb1847966292a671ca7bf


ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: c88d533399fbb1847966292a671ca7bf